In [ ]:
# Imports

import tkinter as tk, ctypes, os, sys, re, json
from tkinter import ttk

dpi = ctypes.windll.shcore.SetProcessDpiAwareness(True)

In [2]:
# Lists and Variables

# Base offsets
BASE = {
    "NTSC": {
        "Character": "80884E6C",
        "Class": "80B54010",
        "Item": "80B5FE10",
        "Skill": "8070EE14",
    },
    "PAL": {
        "Character": "808773EC",
        "Class": "80B55670",
        "Item": "808D2170",
        "Skill": "80701394",
    },
}

# Pull offset data for each section
DESC = json.load(open("Assets/Descriptions.json", "r"))
CHAR = json.load(open("Assets/Characters.json", "r"))
CLASS = json.load(open("Assets/Classes.json", "r"))
ITEM = json.load(open("Assets/Items.json", "r"))
SKILL = json.load(open("Assets/Skills.json", "r"))
LISTS = json.load(open("Assets/Lists.json", "r"))
CODE_DATABASE = json.load(open("Assets/Code_Database.json", "r"))

# Define lists to be used in the GUI
CHAR_LIST = sorted(set(list(CHAR["NTSC"].keys()) + list(CHAR["PAL"].keys())))
CLASS_LIST = list(CLASS["ID"])[1:]
ITEM_LIST = list(ITEM["ID"])[1:]
SECTION_HEADER = ("TkDefaultFont", 10, "bold")

# Define lists that will be used for GUI and code generation
CHAR_STATS = LISTS["CHAR_STATS"]
CHAR_RANKS = LISTS["CHAR_RANKS"]
CLASS_STATS = LISTS["CLASS_STATS"]
ITEM_STATS = LISTS["ITEM_STATS"]
ITEM_DATA = LISTS["ITEM_DATA"]
ITEM_BONUS = LISTS["ITEM_BONUS"]

In [3]:
# UDF

def set_version(ver):
    global VERSION, NTSC_MOD
    if ver == 'NTSC 1.00':
        VERSION = 'NTSC'
        NTSC_MOD = -int('80', 16)
    elif ver == 'NTSC 1.01' or ver == '':
        VERSION = 'NTSC'
        NTSC_MOD = 0
    elif ver == 'PAL':
        VERSION = 'PAL'
        NTSC_MOD = 0

def get_offset(name, data, opt):

    # Check data for Step
    if 'Step' in data:
        if data == 'Item_Step':
            return CHAR['OFFSET']['Item_Step']
        elif data == 'Skill_Step':
            return CHAR['OFFSET']['Skill_Step']

    # Check if the option is 'Char'
    elif opt == 'Char':

        # Get the ID for the character name
        id = CHAR[VERSION][name]

        if data == 'Character':
            # Calculate the offset for the character
            off = int(BASE[VERSION]['Character'], 16) + NTSC_MOD + (CHAR['OFFSET']['Character'] * id)

        else:
            # Calculate the offset for the specific data type
            off = int(BASE[VERSION]['Character'], 16) + NTSC_MOD + (CHAR['OFFSET']['Character'] * id) + CHAR['OFFSET'][data]

        # Convert the offset to a hexadecimal string, remove '0x', convert to uppercase, and pad with zeros
        off = hex(off).replace('0x', '').upper().zfill(8)
        return off
    
    # Check if the option is 'Class'
    elif opt == 'Class':
        # Get the ID for the class name
        id = CLASS['ID'][name]

        if data == 'Class':
            # Calculate the offset for the class
            off = int(BASE[VERSION]['Class'], 16) + NTSC_MOD + (CLASS['OFFSET']['Class'] * id)

        else:
            # Calculate the offset for the specific data type
            off = int(BASE[VERSION]['Class'], 16) + NTSC_MOD + (CLASS['OFFSET']['Class'] * id) + CLASS['OFFSET'][data]

        # Convert the offset to a hexadecimal string, remove '0x', convert to uppercase, and pad with zeros
        off = hex(off).replace('0x', '').upper().zfill(8)
        return off
    
    # Check if the option is 'Item'
    elif opt == 'Item':
        # Get the ID for the item name
        id = ITEM['ID'][name]

        if data == 'Item':
            # Calculate the offset for the item
            off = int(BASE[VERSION]['Item'], 16) + NTSC_MOD + (ITEM['OFFSET']['Item'] * id)

        else:
            # Calculate the offset for the specific data type
            off = int(BASE[VERSION]['Item'], 16) + NTSC_MOD + (ITEM['OFFSET']['Item'] * id) + ITEM['OFFSET'][data]

        # Convert the offset to a hexadecimal string, remove '0x', convert to uppercase, and pad with zeros
        off = hex(off).replace('0x', '').upper().zfill(8)
        return off

def get_char_code(data, kb):

    char_all_code = 'A203F0 00000000'

    # Create code output and start with Keybind
    char_output = []
    char_output.append(kb)

    # Character Select Validation
    char = data['character']
    if not char:
        return "No character selected!"

    if CHAR[VERSION].get(char) == 'Unknown':
        return f"{char} ID unknown in the {VERSION} version of the game. Please report on my discord."

    # Define Step Counts
    item_step = get_offset(char, 'Item_Step', 'Char')
    skill_step = get_offset(char, 'Skill_Step', 'Char')

    #region Class

    char_class = data['class']
    if char_class:

        # Get offsets
        char_class_off = get_offset(char, 'Class', 'Char')
        class_id = get_offset(char_class, 'Class', 'Class')
    
        if char == 'All':
            char_class_code = f'08{char_class_off[-6:]} {class_id}\n20{char_all_code}'
        else:
            char_class_code = f'04{char_class_off[-6:]} {class_id}'
        char_output.append(char_class_code)
    
    #endregion

    #region Items
    for i in range(0,7):
        item = data['items'][i]['item']
        fname = data['items'][i]['forge_name']
        uses = data['items'][i]['uses']
        blessed = data['items'][i]['blessed']
        forged = data['items'][i]['forged']
        mt = data['items'][i]['mt']
        hit = data['items'][i]['hit']
        crit = data['items'][i]['crit']
        wt = data['items'][i]['wt']

        # Get item offsets
        item_off = hex(int(get_offset(char, 'Item', 'Char'), 16) + (item_step * i)).replace('0x', '').upper().zfill(8)
        item_uses_off = hex(int(get_offset(char, 'Item_Uses', 'Char'), 16) + (item_step * i)).replace('0x', '').upper().zfill(8)
        item_status_off = hex(int(get_offset(char, 'Item_Status', 'Char'), 16) + (item_step * i)).replace('0x', '').upper().zfill(8)
        item_forge_off = hex(int(get_offset(char, 'Item_Forge', 'Char'), 16) + (item_step * i)).replace('0x', '').upper().zfill(8)

        # If item is populated
        if item:
            # Get item ID
            item_id = get_offset(item, 'Item', 'Item')

            # Create item code and add to output
            if char == 'All':
                char_item_code = f'08{item_off[-6:]} {item_id}\n20{char_all_code}'
            else:
                char_item_code = f'04{item_off[-6:]} {item_id}'
            char_output.append(char_item_code)

        # If uses or item is populated
        if uses or item:
            if uses == '':
                uses = 0

            # Error handling for uses
            try:
                uses = int(uses)

                # Check if uses is within range
                if uses > 255:
                    return f'Error: Uses for {item} is too high! Please enter a value between 0 and 255.'

                # Default to 80 uses if not populated
                if uses == 0:
                    uses = 80

                if char == 'All':
                    char_item_uses_code = f'08{item_uses_off[-6:]} 000000{uses}\n00{char_all_code}'
                else:
                    char_item_uses_code = f'00{item_uses_off[-6:]} 000000{hex(uses).replace('0x', '').zfill(2).upper()}'
                char_output.append(char_item_uses_code)

            # Error handling for uses
            except ValueError:
                return f'Error: Uses for {item} is not a number! Please enter a value between 0 and 255.'
        
        # If blessed = True or forged = True or item is populated
        if blessed or forged or item:

            # Determine equip status
            sts = 0

            if blessed:
                sts += int('10', 16)

            if forged:
                sts += int('20', 16)
            
            # Create status code and add to output
            status = hex(sts).replace('0x', '').zfill(2).upper()
            if char == 'All':
                char_status_code = f'08{item_status_off[-6:]} 000000{status}\n00{char_all_code}'
            else:
                char_status_code = f'00{item_status_off[-6:]} 000000{status}'
            char_output.append(char_status_code)

            # If forged = True
            if forged:
                # Create variable for forge name
                fname_hex = ''

                # Error handling for forge name
                if len(fname) > 26:
                        return f'Error: Forge Name for {item} is too long! Please enter a name with 26 characters or less.'
                elif fname and 0 < len(fname) <= 26:
                    # If forge name is populated, convert to hex
                    for c in fname:
                        fname_hex += format(ord(c), "x").zfill(2)
                else:
                    # If forge name does not meet requirements, default forge name to item name
                    for c in item:
                        fname_hex += format(ord(c), "x").zfill(2)
                
                # Pad forge name to 60 digits (26 characters in hex)
                fname_hex = fname_hex.ljust(52, '0').upper()

                # Create forge name code and add to output
                j = 0

                # Loop through forge name and add to output
                for k in range(0, 7):
                    # First set is 16 bytes, so it needs to be handled differently
                    if k == 0:
                        fname_offset = hex(int(item_off, 16) + 6).replace('0x', '').zfill(8).upper()
                        if char == 'All':
                            char_fname_code = f'08{fname_offset[-6:]} 0000{fname_hex[:4]}\n10{char_all_code}'
                        else:
                            char_fname_code = f'02{fname_offset[-6:]} 0000{fname_hex[:4]}'
                        char_output.append(char_fname_code)

                    # The rest of the sets are 32 bytes
                    else:
                        fname_offset = hex(int(item_off, 16) + 8 + j).replace('0x', '').zfill(8).upper()
                        if char == 'All':
                            char_fname_code = f'08{fname_offset[-6:]} {fname_hex[4+(j*2):12+(j*2)]}\n20{char_all_code}'
                            if fname_hex[4+(j*2):12+(j*2)] != '00000000':
                                char_output.append(char_fname_code)
                        else:
                            char_fname_code = f'04{fname_offset[-6:]} {fname_hex[4+(j*2):12+(j*2)]}'
                            if char_fname_code[-8:] != '00000000':
                                char_output.append(char_fname_code)

                        j += 4

                # If mt, hit, wt, or crit is populated
                if mt or hit or wt or crit:

                    # Default values if not populated
                    if mt == '':
                        mt = 0
                    if hit == '':
                        hit = 0
                    if wt == True:
                        wt = 'E0'
                    else:
                        wt = '00'
                    if crit == '':
                        crit = 0

                    # Error handling for mt, hit, wt, and crit
                    try:
                        # Validation
                        mt = int(mt)
                        hit = int(hit)
                        crit = int(crit)
                        if mt > 255 or hit > 255 or crit > 255:
                            return f'Error: Stat for {item} is too high! Please enter a value between 0 and 255.'
                        
                        # Create stat codes and add to output, 16 bytes each
                        if char == 'All':
                            mt_hit_code = f'08{item_forge_off[-6:]} 0000{hex(mt).replace("0x", "").zfill(2).upper()}{hex(hit).replace("0x", "").zfill(2).upper()}\n10{char_all_code}'
                            crit_wt_off = hex(int(item_forge_off, 16) + 2).replace("0x", "").zfill(8).upper()
                            crit_wt_code = f'08{crit_wt_off[-6:]} 0000{hex(crit).replace("0x", "").zfill(2).upper()}{wt}\n10{char_all_code}'

                        else:
                            mt_hit_code = f'02{item_forge_off[-6:]} 0000{hex(mt).replace('0x', '').zfill(2).upper()}{hex(hit).replace('0x', '').zfill(2).upper()}'
                            crit_wt_off = hex(int(item_forge_off, 16) + 2).replace('0x', '').zfill(8).upper()
                            crit_wt_code = f'02{crit_wt_off[-6:]} 0000{hex(crit).replace('0x', '').zfill(2).upper()}{wt}'

                        char_output.append(mt_hit_code)
                        char_output.append(crit_wt_code)
                    
                    # Error handling for mt, hit, wt, and crit
                    except ValueError:
                        return f'Error: Stat for {item} is not a number! Please enter a value between 0 and 255.'
    
    #endregion

    #region Stats

    for chstat, char_stat in enumerate(CHAR_STATS):

            # Get data input
            char_stat_input = data['stats'][chstat]
            char_stat_offset = get_offset(char, char_stat, 'Char')

            if char_stat_input:
                try:
                    char_stat_num = int(char_stat_input)
                    if char_stat == 'Level' and char_stat_num > 20:
                        return f'Error: Stat for {char_stat.replace("_", " ")} is too high! Please enter a value between 0 and 20.'
                    elif char_stat == 'EXP' and char_stat_num > 99:
                        return f'Error: Stat for {char_stat.replace("_", " ")} is too high! Please enter a value between 0 and 99.'
                    if char_stat_num > 255:
                        return f'Error: Stat for {char_stat.replace("_", " ")} is too high! Please enter a value between 0 and 255.'
                    
                    if char == 'All':
                        char_stat_code = f'08{char_stat_offset[-6:]} 000000{hex(char_stat_num).replace("0x", "").zfill(2).upper()}\n00{char_all_code}'

                    else:
                        char_stat_code = f'00{char_stat_offset[-6:]} 000000{hex(char_stat_num).replace("0x", "").zfill(2).upper()}'

                    char_output.append(char_stat_code)
                
                except ValueError:
                    return f'Error: Stat for {char_stat} is not a number! Please enter a value between 0 and 255.'

    #endregion

    #region Weapon Ranks

    for chwr, char_rank in enumerate(CHAR_RANKS):

        # Get data input
        char_rank_input = data['ranks'][chwr]
        char_rank_offset = get_offset(char, char_rank, 'Char')

        # If data is populated
        if char_rank_input:
            if char_rank_input == 'SS':
                rank = '014B'
            elif char_rank_input == 'S':
                rank = '00FB'
            elif char_rank_input == 'A':
                rank = '00B5'
            elif char_rank_input == 'B':
                rank = '0079'
            elif char_rank_input == 'C':
                rank = '0047'
            elif char_rank_input == 'D':
                rank = '001F'
            elif char_rank_input == 'E':
                rank = '0001'
            else:
                return f'Error: Invalid weapon rank for {char_rank.replace('_', ' ')}! Please select a valid rank.'

            if char == 'All':
                char_rank_code = f'08{char_rank_offset[-6:]} 0000{rank}\n10{char_all_code}'

            else:
                char_rank_code = f'02{char_rank_offset[-6:]} 0000{rank}'

            char_output.append(char_rank_code)

    #endregion

    # Add end code to output
    char_output.append('E0000000 80008000')

    # If only kb and end code, return no changes made
    if len(char_output) == 2:
        return "No changes made!"
    else:
        return "\n".join(char_output)

def get_class_code(data):

    class_all_code = 'AA011C 00000000'

    # Create code output and append start code
    class_output = []

    if VERSION == 'NTSC':
        class_output.append('20B54158 8070F8BC')
    elif VERSION == 'PAL':
        class_output.append('20B58CF8 80701E3C')

    # Class Select Validation
    cls = data['class']
    if not cls:
        return "No class selected!"

    #region Promote

    promote = data['promote']
    if promote:
        class_promote_off = get_offset(cls, 'Next_Class', 'Class')
        class_id = get_offset(promote, 'Class', 'Class')

        if cls == 'All':
            class_promote_code = f'08{class_promote_off[-6:]} {class_id}\n20{class_all_code}'
        else:
            class_promote_code = f'04{class_promote_off[-6:]} {class_id}'
        class_output.append(class_promote_code)

    #endregion

    #region Weapon Ranks

    min_ranks = ['Min_' + i for i in CHAR_RANKS]
    max_ranks = ['Max_' + i for i in CHAR_RANKS]
    class_ranks = min_ranks + max_ranks

    for clwr, class_rank in enumerate(class_ranks):

        # Get data input
        class_rank_input = data['ranks'][clwr]
        class_rank_offset = get_offset(cls, class_rank, 'Class')

        # If data is populated
        if class_rank_input:
            if class_rank_input == 'SS':
                rank = '014B'
            elif class_rank_input == 'S':
                rank = '00FB'
            elif class_rank_input == 'A':
                rank = '00B5'
            elif class_rank_input == 'B':
                rank = '0079'
            elif class_rank_input == 'C':
                rank = '0047'
            elif class_rank_input == 'D':
                rank = '001F'
            elif class_rank_input == 'E':
                rank = '0001'
            else:
                return f'Error: Invalid weapon rank for {class_rank.replace('_', ' ')}! Please select a valid rank.'

            if cls == 'All':
                class_rank_code = f'08{class_rank_offset[-6:]} 0000{rank}\n10{class_all_code}'
            else:
                class_rank_code = f'02{class_rank_offset[-6:]} 0000{rank}'
            class_output.append(class_rank_code)
    
    #endregion

    #region Stats

    for clstat, class_stat in enumerate(CLASS_STATS):

        class_stat_input = data['stats'][clstat]
        class_stat_offset = get_offset(cls, class_stat, 'Class')

        if class_stat_input:

            try:

                class_stat_num = int(class_stat_input)
                if class_stat_num > 255:
                    return f'Error: Stat for {class_stat.replace('_', ' ')} is too high! Please enter a value between 0 and 255.'
                
                if cls == 'All':
                    class_stat_code = f'08{class_stat_offset[-6:]} 000000{hex(class_stat_num).replace("0x", "").zfill(2).upper()}\n00{class_all_code}'
                else:
                    class_stat_code = f'00{class_stat_offset[-6:]} 000000{hex(class_stat_num).replace("0x", "").zfill(2).upper()}'
                class_output.append(class_stat_code)
            
            # Error handling for stats
            except ValueError:
                return f'Error: Stat for {class_stat} is not a number! Please enter a value between 0 and 255.'

    #endregion

    # Add end code to output
    class_output.append('E0000000 80008000')

    # If only start and end code, return no changes made
    if len(class_output) == 2:
        return "No changes made!"
    else:
        return "\n".join(class_output)

def get_item_code(data):

    item_all_code = 'CA0050 00000000'

    # Create code output and append start code
    item_output = []
    
    if VERSION == 'NTSC':
        item_output.append('20B54158 8070F8BC')
    elif VERSION == 'PAL':
        item_output.append('20B58CF8 80701E3C')

    # Item Select Validation
    item = data['item']
    if not item:
        return "No item selected!"

    #region Item Data

    for item_data in ITEM_DATA:

        # Get data input
        item_data_input = data['data'][item_data]
        item_data_offset = get_offset(item, item_data, 'Item')

        # If data is populated
        if item_data_input:
            # Determine attack type
            if item_data == 'Attack_Type':
                if item_data_input == 'STR':
                    str_mag = '00'
                elif item_data_input == 'MAG':
                    str_mag = '06'
                else:
                    return f'Error: Invalid attack type for {item}! Please select a valid type.'
                
                if item == 'All':
                    item_data_code = f'08{item_data_offset[-6:]} 000000{str_mag}\n00{item_all_code}'
                else:
                    item_data_code = f'00{item_data_offset[-6:]} 000000{str_mag}'
                item_output.append(item_data_code)

            # Determine weapon rank
            elif item_data == 'Weapon_Rank':
                if item_data_input == 'SS':
                    rank = '014B'
                elif item_data_input == 'S':
                    rank = '00FB'
                elif item_data_input == 'A':
                    rank = '00B5'
                elif item_data_input == 'B':
                    rank = '0079'
                elif item_data_input == 'C':
                    rank = '0047'
                elif item_data_input == 'D':
                    rank = '001F'
                elif item_data_input == 'E':
                    rank = '0001'
                else:
                    return f'Error: Invalid weapon rank for {item}! Please select a valid rank.'
                
                if item == 'All':
                    item_data_code = f'08{item_data_offset[-6:]} 0000{rank}\n10{item_all_code}'
                else:
                    item_data_code = f'02{item_data_offset[-6:]} 0000{rank}'
                item_output.append(item_data_code)

            # Determine EXP gain
            elif item_data == 'EXP Gain':
                if item == 'All':
                    item_data_code = f'08{item_data_offset[-6:]} 000000{hex(int(item_data_input)).replace("0x", "").zfill(2).upper()}\n00{item_all_code}'
                else:
                    item_data_code = f'00{item_data_offset[-6:]} 000000{hex(int(item_data_input)).replace("0x", "").zfill(2).upper()}'
                item_output.append(item_data_code)

            # Determine Unlock
            elif item_data == 'Unlock':
                if item_data_input:
                    unlock = '00'
                    if item == 'All':
                        item_data_code = f'08{item_data_offset[-6:]} 000000{unlock}\n00{item_all_code}'
                    else:
                        item_data_code = f'00{item_data_offset[-6:]} 000000{unlock}'
                    item_output.append(item_data_code)

            # Determine Infinite and Brave
            elif item_data in ['Infinite', 'Brave']:
                if item_data_input:
                    inf_brave = '01'
                    if item == 'All':
                        item_data_code = f'08{item_data_offset[-6:]} 000000{inf_brave}\n00{item_all_code}'
                    else:
                        item_data_code = f'00{item_data_offset[-6:]} 000000{inf_brave}'
                    item_output.append(item_data_code)

            # Determine Char Unlock
            elif item_data == 'Char_Unlock':
                if item_data_input:
                    c_unlock = '0000'
                    if item == 'All':
                        item_data_code = f'08{item_data_offset[-6:]} 0000{c_unlock}\n10{item_all_code}'
                    else:
                        item_data_code = f'02{item_data_offset[-6:]} 0000{c_unlock}'
                    item_output.append(item_data_code)

            # Determine Heal
            elif item_data == 'Heal':
                if item_data_input:
                    heal = '10'
                    if item == 'All':
                        item_data_code = f'08{item_data_offset[-6:]} 000000{heal}\n00{item_all_code}'
                    else:
                        item_data_code = f'00{item_data_offset[-6:]} 000000{heal}'
                    item_output.append(item_data_code)
    
    #endregion

    #region Item Stats

    for istat, item_stat in enumerate(ITEM_STATS):

        # Get data input and offset for item stat
        item_stat_input = data['stats'][istat]
        item_stat_offset = get_offset(item, item_stat, 'Item')

        # If data is populated
        if item_stat_input:
            try:
                # Validation
                item_stat_num = int(item_stat_input)
                if item_stat_num > 255:
                    return f'Error: Stat for {item_stat.replace("_", " ")} is too high! Please enter a value between 0 and 255.'
                
                if item == 'All':
                    item_stat_code = f'08{item_stat_offset[-6:]} 000000{hex(item_stat_num).replace("0x", "").zfill(2).upper()}\n00{item_all_code}'
                else:
                    item_stat_code = f'00{item_stat_offset[-6:]} 000000{hex(item_stat_num).replace("0x", "").zfill(2).upper()}'
                item_output.append(item_stat_code)

            # Error handling for item stats
            except ValueError:
                return f'Error: Stat for {item_stat} is not a number! Please enter a value between 0 and 255.'

    #endregion

    #region Item Equip Bonuses

    for ibonus, bonus in enumerate(ITEM_BONUS):

        # Get data input and offset for equip bonus
        item_bonus_input = data['bonuses'][ibonus]
        item_bonus_offset = get_offset(item, bonus, 'Item')

        # If data is populated
        if item_bonus_input:
            try:
                # Validation
                bonus_num = int(item_bonus_input)
                if bonus_num > 255:
                    return f'Error: Equip Bonus for {bonus.replace("_", " ")} is too high! Please enter a value between 0 and 255.'
                
                if item == 'All':
                    item_bonus_code = f'08{item_bonus_offset[-6:]} 000000{hex(bonus_num).replace("0x", "").zfill(2).upper()}\n00{item_all_code}'
                else:
                    item_bonus_code = f'00{item_bonus_offset[-6:]} 000000{hex(int(item_bonus_input)).replace("0x", "").zfill(2).upper()}'
                item_output.append(item_bonus_code)

            # Error handling for equip bonuses
            except ValueError:
                return f'Error: Equip Bonus for {bonus} is not a number! Please enter a value between 0 and 255.'
    
    #endregion

    # Add end code to output
    item_output.append('E0000000 80008000')

    # If only start and end code, return no changes made
    if len(item_output) == 2:
        return "No changes made!"
    else:
        return "\n".join(item_output)

def get_keybind_code(data):
    val = 0
    if data['controller'] == '':
        if VERSION == 'NTSC':
            return '20B54158 8070F8BC'
        elif VERSION == 'PAL':
            return '20B58CF8 80701E3C'
    elif data['controller'] == 'Wiimote+Nunchuck':
        # Left
        if data['keys'][0]:
            val += int('1', 16)
        # Right
        if data['keys'][1]:
            val += int('2', 16)
        # Up
        if data['keys'][2]:
            val += int('8', 16)
        # Down
        if data['keys'][3]:
            val += int('4', 16)
        # A
        if data['keys'][4]:
            val += int('800', 16)
        # B
        if data['keys'][5]:
            val += int('400', 16)
        # C
        if data['keys'][6]:
            val += int('4000', 16)
        # Z
        if data['keys'][7]:
            val += int('2000', 16)
        # 1
        if data['keys'][8]:
            val += int('200', 16)
        # 2
        if data['keys'][9]:
            val += int('100', 16)
        # Plus
        if data['keys'][10]:
            val += int('10', 16)
        # Minus
        if data['keys'][11]:
            val += int('1000', 16)
        return f'28 {hex(val).replace('0x', '').zfill(8)}'

    elif data['controller'] == 'Classic Controller':
        val = 0
        # Left
        if data['keys'][0]:
            val += int('2', 16)
        # Right
        if data['keys'][1]:
            val += int('8000', 16)
        # Up
        if data['keys'][2]:
            val += int('1', 16)
        # Down
        if data['keys'][3]:
            val += int('4000', 16)
        # A
        if data['keys'][4]:
            val += int('10', 16)
        # B
        if data['keys'][5]:
            val += int('40', 16)
        # X
        if data['keys'][6]:
            val += int('8', 16)
        # Y
        if data['keys'][7]:
            val += int('20', 16)
        # ZL
        if data['keys'][8]:
            val += int('80', 16)
        # ZR
        if data['keys'][9]:
            val += int('4', 16)
        # L
        if data['keys'][10]:
            val += int('2000', 16)
        # R
        if data['keys'][11]:
            val += int('200', 16)
        # Plus
        if data['keys'][12]:
            val += int('400', 16)
        # Minus
        if data['keys'][13]:
            val += int('1000', 16)
        
        if VERSION == 'NTSC':
            return f'283D79BA {hex(val).replace('0x', '').zfill(8)}'
        elif VERSION == 'PAL':
            return f'283D035A {hex(val).replace('0x', '').zfill(8)}'
    
    elif data['controller'] == 'GameCube Controller':
        # Left
        if data['keys'][0]:
            val += int('1', 16)
        # Right
        if data['keys'][1]:
            val += int('2', 16)
        # Up
        if data['keys'][2]:
            val += int('8', 16)
        # Down
        if data['keys'][3]:
            val += int('4', 16)
        # A
        if data['keys'][4]:
            val += int('100', 16)
        # B
        if data['keys'][5]:
            val += int('200', 16)
        # X
        if data['keys'][6]:
            val += int('400', 16)
        # Y
        if data['keys'][7]:
            val += int('800', 16)
        # Z
        if data['keys'][8]:
            val += int('10', 16)
        # L
        if data['keys'][9]:
            val += int('40', 16)
        # R
        if data['keys'][10]:
            val += int('20', 16)
        # Start
        if data['keys'][11]:
            val += int('1000', 16)
        if VERSION == 'NTSC':
            return f'283D7928 {hex(val).replace('0x', '').zfill(8)}'
        elif VERSION == 'PAL':
            return f'283D02C8 {hex(val).replace('0x', '').zfill(8)}'

In [4]:
# GUI

class CodeGeneratorGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("FE:RD Code Creator")

        # Set no resize
        self.root.resizable(False, False)

        icon_path = "Assets/FE-RD.ico"
        self.root.iconbitmap(icon_path)

        # Set dark mode colors
        style = ttk.Style()
        style.theme_use("clam")
        style.configure("TNotebook", background="#000000", foreground="#ffffff")
        style.configure("TNotebook.Tab", background="#000000", foreground="#ffffff")
        style.map("TNotebook.Tab", background=[("selected", "#2e2e2e")])

        # Configure text color for various widgets
        style.configure("TFrame", background="#2e2e2e")
        style.configure("TLabel", background="#2e2e2e", foreground="#ffffff")
        style.configure("TEntry", fieldbackground="#ffffff", foreground="#000000")
        style.configure("TCombobox", fieldbackground="#ffffff", foreground="#000000")
        style.map("TCombobox",fieldbackground=[("readonly", "#ffffff")],foreground=[("readonly", "#000000")],)
        style.configure("TCheckbutton", background="#2e2e2e", foreground="#000000")

        self.notebook = ttk.Notebook(root)
        self.notebook.pack(fill="both", expand=True)

        self.options_tab()
        self.character_tab()
        self.class_tab()
        self.items_tab()
        self.database_tab()

    def copy_and_close(self, code, message_window):
        self.root.clipboard_clear()
        self.root.clipboard_append(code)
        message_window.destroy()

    def output_code(self, code):
        # Create a new window for the message box
        message_window = tk.Toplevel(self.root)
        message_window.title("Code")
        message_window.configure(bg="#2e2e2e")

        # Add a label to display the output
        output_label = ttk.Label(message_window, text=code, justify="center", wraplength=300)
        output_label.pack(padx=10, pady=10)

        match = re.search(r'Code:\n((?:.*\n*)+)', code)
        if match:
            code_part = match.group(1).strip()
        else:
            code_part = code

        error_list = [
            "No character selected!",
            "No class selected!",
            "No item selected!",
            "No changes made!"
            ]
        if code not in error_list and "Error:" not in code:
            # Add a button to copy to clipboard
            copy_button = ttk.Button(message_window, text="Copy to Clipboard", command=lambda: self.copy_and_close(code_part, message_window))
            copy_button.pack(pady=5)

    def options_tab(self):
        options_tab = ttk.Frame(self.notebook)
        self.notebook.add(options_tab, text="Options")

        #region Separators
        ttk.Separator(options_tab, orient="horizontal").grid(row=1, column=0, columnspan=99, sticky="ew")
        ttk.Separator(options_tab, orient="vertical").grid(row=0, column=2, rowspan=4, sticky="ns")
        ttk.Separator(options_tab, orient="vertical").grid(row=0, column=4, rowspan=4, sticky="ns")
        ttk.Separator(options_tab, orient="horizontal").grid(row=3, column=0, columnspan=99, sticky="ew")
        #endregion

        #region Controller

        self.keybinds = {
            "": "",
            # "Wiimote+Nunchuck": [
            #     "Left",
            #     "Right",
            #     "Up",
            #     "Down",
            #     "A",
            #     "B",
            #     "C",
            #     "Z",
            #     "1",
            #     "2",
            #     "+",
            #     "-"
            # ],
            "Classic Controller": [
                "Left",
                "Right",
                "Up",
                "Down",
                "A",
                "B",
                "X",
                "Y",
                "ZL",
                "ZR",
                "L",
                "R",
                "+",
                "-",
            ],
            "GameCube Controller": [
                "Left",
                "Right",
                "Up",
                "Down",
                "A",
                "B",
                "X",
                "Y",
                "Z",
                "L",
                "R",
                "Start",
            ],
        }

        controller_frame = ttk.Frame(options_tab)
        controller_frame.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")

        ttk.Label(controller_frame, text="Controller:", font=SECTION_HEADER).grid(row=0, column=0, padx=5)
        self.controller = ttk.Combobox(controller_frame, values=list(self.keybinds.keys()))
        self.controller.grid(row=0, column=1)
        self.controller.bind("<<ComboboxSelected>>", self.update_checkboxes)

        # Checkboxes Section
        self.checkboxes = []
        self.checkbox_frame = ttk.Frame(options_tab)
        self.checkbox_frame.grid(row=2, column=0, columnspan=2, padx=10, pady=10)

        #endregion

        #region Version

        version_frame = ttk.Frame(options_tab)
        version_frame.grid(row=0, column=3, padx=10, pady=10, sticky="nsew")

        ttk.Label(version_frame, text="Version:", font=SECTION_HEADER).grid(row=0, column=0, padx=5)
        self.version = ttk.Combobox(version_frame, values=["NTSC 1.00", "NTSC 1.01", "PAL"])
        self.version.grid(row=0, column=1)

        #endregion

        #region Options Info

        options_info = ttk.Frame(options_tab)
        options_info.grid(row=2, column=3, padx=10, pady=10, sticky="nsew")

        opt_desc = DESC["Option_Desc"]

        ttk.Label(options_info, text=opt_desc, wraplength=1000, justify="left").grid(row=0, column=0)

        #endregion

        #region Generate Button

        gen__all_button = ttk.Button(options_tab, text="Generate All Codes", command=self.generate_all)
        gen__all_button.grid(row=4, column=0, columnspan=99, sticky='nsew')

        #endregion

    def update_checkboxes(self, event):
        for checkbox in self.checkbox_frame.winfo_children():
            checkbox.destroy()
        self.checkboxes = []

        selected_option = self.controller.get()
        if selected_option == "":
            pass
        elif selected_option in self.keybinds:
            length = len(self.keybinds[selected_option])
            ttk.Label(self.checkbox_frame, text="Buttons", font=SECTION_HEADER).grid(row=0, column=0, rowspan=length, padx=5)
            ttk.Separator(self.checkbox_frame, orient="vertical").grid(row=0, column=1, rowspan=length, sticky="ns")
            for i, option in enumerate(self.keybinds[selected_option]):
                ttk.Label(self.checkbox_frame, text=option).grid(row=i, column=3, padx=5, sticky="e")
                var = tk.BooleanVar()
                checkbox = ttk.Checkbutton(self.checkbox_frame, variable=var)
                checkbox.grid(row=i, column=4)
                self.checkboxes.append(var)

    def character_tab(self):

        char_tab = ttk.Frame(self.notebook)
        self.notebook.add(char_tab, text="Character")

        #region Separators
        ttk.Separator(char_tab, orient="vertical").grid(row=0, column=3, sticky="ns")
        ttk.Separator(char_tab, orient="horizontal").grid(row=1, column=0, columnspan=99, sticky="ew")
        ttk.Separator(char_tab, orient="vertical").grid(row=2, column=1, sticky="ns")
        ttk.Separator(char_tab, orient="vertical").grid(row=2, column=3, sticky="ns")
        ttk.Separator(char_tab, orient="horizontal").grid(row=3, column=0, columnspan=99, sticky="ew")
        #endregion

        #region Character and Class

        # Character Select Frame
        char_frame = ttk.Frame(char_tab)
        char_frame.grid(row=0, column=0, sticky="nsew", columnspan=3, padx=10, pady=10)

        ttk.Label(char_frame, text="Character:", font=SECTION_HEADER).grid(row=0, column=0)
        self.char_select = ttk.Combobox(char_frame, values=CHAR_LIST)
        self.char_select.grid(row=0, column=1, padx=10)

        # Class Select Frame
        char_class_frame = ttk.Frame(char_tab)
        char_class_frame.grid(row=0, column=4, sticky="nsew", columnspan=4, padx=10, pady=10)

        ttk.Label(char_class_frame, text="Class:", font=SECTION_HEADER).grid(row=0, column=0)
        self.char_class = ttk.Combobox(char_class_frame, values=CLASS_LIST, width=40)
        self.char_class.grid(row=0, column=1, padx=10)

        # endregion

        #region Items Section
        self.char_items = []
        char_items_frame = ttk.Frame(char_tab)
        char_items_frame.grid(row=2, column=5, padx=10, pady=10, sticky='nsew')

        # Item table
        char_item_table = ttk.Frame(char_items_frame)
        char_item_table.grid(row=0, column=0, sticky='n')

        headers = [
            "Items",
            "Uses",
            "Forged Name",
            "Mt",
            "Hit",
            "Crit",
            "Weightless",
            "Forged",
            "Blessed"
        ]
        for i, header in enumerate(headers):
            if header in ['Blessed', 'Forged', 'Weightless']:
                ttk.Label(char_item_table, text=header, font=SECTION_HEADER).grid(row=0, column=i, padx=5, pady=5)
            else:
                ttk.Label(char_item_table, text=header, font=SECTION_HEADER).grid(row=0, column=i, pady=5)

        for row in range(1, 8):
            char_item_row = []
            char_item_combobox = ttk.Combobox(char_item_table, values=ITEM_LIST, width=25)
            char_item_combobox.grid(row=row, column=0, padx=1, pady=1)
            char_item_row.append(char_item_combobox)

            for col in range(1, 9):
                if col == 6 or col == 7 or col == 8:
                    char_item_var = tk.BooleanVar(value=False)
                    if col == 8:
                        blessed_checkbox = ttk.Checkbutton(char_item_table, text="", variable=char_item_var)
                        blessed_checkbox.grid(row=row, column=col)
                    elif col == 7:
                        forged_checkbox = ttk.Checkbutton(char_item_table, text="", variable=char_item_var)
                        forged_checkbox.grid(row=row, column=col)
                    elif col == 6:
                        wt_checkbox = ttk.Checkbutton(char_item_table, text="", variable=char_item_var)
                        wt_checkbox.grid(row=row, column=col)
                    char_item_row.append(char_item_var)
                else:
                    if col == 2:
                        char_item_entry = ttk.Entry(char_item_table, width=20)
                        char_item_entry.grid(row=row, column=col, padx=1, pady=1)
                        char_item_row.append(char_item_entry)
                    else:
                        char_item_entry = ttk.Entry(char_item_table, width=5)
                        char_item_entry.grid(row=row, column=col, padx=1, pady=1)
                        char_item_row.append(char_item_entry)

            self.char_items.append(char_item_row)

        # Item Description
        char_desc_frame = ttk.Frame(char_items_frame)
        char_desc_frame.grid(row=1, column=0, sticky='nsew')

        char_desc = DESC['Char_Tab_Desc']

        ttk.Label(char_desc_frame, text=char_desc, wraplength=1000, justify='left').grid(row=0, column=0, padx=5, pady=5)

        # endregion

        #region Character Stats

        char_stat_frame = ttk.Frame(char_tab)
        char_stat_frame.grid(row=2, column=0, padx=10, pady=10, sticky='nsew')

        ttk.Label(char_stat_frame, text="Stats", font=SECTION_HEADER).grid(row=0, column=0, columnspan=2, pady=5)
        self.char_stats = []

        for i, stat in enumerate(CHAR_STATS):
            ttk.Label(char_stat_frame, text=stat).grid(row=i+1, column=0, padx=5, sticky="e")
            char_stat_entry = ttk.Entry(char_stat_frame, width=5)
            char_stat_entry.grid(row=i+1, column=1, pady=1)
            self.char_stats.append(char_stat_entry)

        #endregion

        #region Character Weapon Ranks

        char_rank_frame = ttk.Frame(char_tab)
        char_rank_frame.grid(row=2, column=2, padx=10, pady=10, sticky='nsew')

        ttk.Label(char_rank_frame, text="Weapon Ranks", font=SECTION_HEADER).grid(row=0, column=3, columnspan=2, pady=5)
        self.char_ranks = []

        for i, rank in enumerate(CHAR_RANKS):
            ttk.Label(char_rank_frame, text=rank.replace('_', ' ')).grid(row=i + 1, column=3, padx=5, sticky="e")
            char_ranks_combobox = ttk.Combobox(char_rank_frame, values=["SS", "S", "A", "B", "C", "D", "E"], width=5)
            char_ranks_combobox.grid(row=i + 1, column=4, pady=1)
            self.char_ranks.append(char_ranks_combobox)
        
        # endregion

        # Generate Button
        char_button = ttk.Button(char_tab, text="Generate", command=self.generate_character)
        char_button.grid(row=4, column=0, columnspan=99, sticky="nsew")

    def generate_character(self):
        character_data = {
            "character": self.char_select.get(),
            "class": self.char_class.get(),
            "stats": [stat.get() for stat in self.char_stats],
            "ranks": [rank.get() for rank in self.char_ranks],
            "items": [
                {
                    "item": row[0].get(),
                    "uses": row[1].get(),
                    "forge_name": row[2].get(),
                    "mt": row[3].get(),
                    "hit": row[4].get(),
                    "crit": row[5].get(),
                    "wt": row[6].get(),
                    "forged": row[7].get(),
                    "blessed": row[8].get(),
                }
                for row in self.char_items
            ]
        }

        keybinds_data = {
            "controller": self.controller.get(),
            "keys": [key.get() for key in self.checkboxes],
        }

        version = self.version.get()
        set_version(version)

        key_code = get_keybind_code(keybinds_data)
        output = get_char_code(character_data, key_code)
        self.output_code(output)

    def class_tab(self):
        class_tab = ttk.Frame(self.notebook)
        self.notebook.add(class_tab, text="Class")

        #region Separators
        ttk.Separator(class_tab, orient="vertical").grid(row=0, column=5, sticky="ns")
        ttk.Separator(class_tab, orient="horizontal").grid(row=1, column=0, columnspan=99, sticky="ew")
        ttk.Separator(class_tab, orient="horizontal").grid(row=3, column=0, columnspan=99, sticky="ew")
        ttk.Separator(class_tab, orient="vertical").grid(row=2, column=1, rowspan=99, sticky="ns")
        ttk.Separator(class_tab, orient="vertical").grid(row=2, column=3, rowspan=99, sticky="ns")
        ttk.Separator(class_tab, orient="vertical").grid(row=2, column=5, rowspan=99, sticky="ns")
        #endregion

        #region Class and Promote section

        # Class Select Section
        class_frame = ttk.Frame(class_tab)
        class_frame.grid(row=0, column=0, padx=10, pady=10, columnspan=5, sticky="nsew")

        ttk.Label(class_frame, text="Class:", font=SECTION_HEADER).grid(row=0, column=0)
        self.class_select = ttk.Combobox(class_frame, values=['All'] + CLASS_LIST, width=40)
        self.class_select.grid(row=0, column=1, padx=10)

        # Promotion Section
        class_promote_frame = ttk.Frame(class_tab)
        class_promote_frame.grid(row=0, column=6, padx=10, pady=10, columnspan=99, sticky="nsew")

        ttk.Label(class_promote_frame, text="Next Class:", font=SECTION_HEADER).grid(row=0, column=3)
        self.class_promote = ttk.Combobox(class_promote_frame, values=CLASS_LIST, width=40)
        self.class_promote.grid(row=0, column=4, padx=10)

        # endregion

        #region Min Weapon Ranks

        # Create frame for weapon ranks and stats

        class_min_rank_frame = ttk.Frame(class_tab)
        class_min_rank_frame.grid(row=2, column=0, padx=10, pady=10, sticky='n')

        # Min Weapon Ranks Section
        ttk.Label(class_min_rank_frame, text="Min Weapon Ranks", font=SECTION_HEADER).grid(row=0, column=0, pady=5, columnspan=2)

        min_weapon_ranks = ['Min_' + i for i in CHAR_RANKS]
        self.class_min_ranks = []

        for i, rank in enumerate(min_weapon_ranks):
            ttk.Label(class_min_rank_frame, text=rank.replace('_', ' ')).grid(row=i + 1, column=0, padx=5, sticky="e")
            class_min_rank_combobox = ttk.Combobox(class_min_rank_frame, values=["SS", "S", "A", "B", "C", "D", "E"], width=5)
            class_min_rank_combobox.grid(row=i + 1, column=1, pady=1)
            self.class_min_ranks.append(class_min_rank_combobox)

        #endregion

        #region Max Weapon Ranks

        class_max_rank_frame = ttk.Frame(class_tab)
        class_max_rank_frame.grid(row=2, column=2, padx=10, pady=10, sticky='n')

        ttk.Label(class_max_rank_frame, text="Max Weapon Ranks", font=SECTION_HEADER).grid(row=0, column=0, pady=5, columnspan=2)

        max_weapon_ranks = ['Max_' + i for i in CHAR_RANKS]
        self.class_max_ranks = []

        for i, rank in enumerate(max_weapon_ranks):
            ttk.Label(class_max_rank_frame, text=rank.replace('_', ' ')).grid(row=i + 1, column=0, padx=5, sticky="e")
            class_max_rank_combobox = ttk.Combobox(class_max_rank_frame, values=["SS", "S", "A", "B", "C", "D", "E"], width=5)
            class_max_rank_combobox.grid(row=i + 1, column=1, pady=1)
            self.class_max_ranks.append(class_max_rank_combobox)

        #endregion

        #region Stats

        class_stats_frame = ttk.Frame(class_tab)
        class_stats_frame.grid(row=2, column=4, padx=10, pady=10, sticky='n')

        ttk.Label(class_stats_frame, text="Stats", font=SECTION_HEADER).grid(row=0, column=0, pady=5, columnspan=2)

        self.class_stats = []

        for i, stat in enumerate(CLASS_STATS):
            ttk.Label(class_stats_frame, text=stat.replace('_', ' ')).grid(row=i + 1, column=0, padx=5, sticky="e")
            class_stats_entry = ttk.Entry(class_stats_frame, width=5)
            class_stats_entry.grid(row=i + 1, column=1, pady=1)
            self.class_stats.append(class_stats_entry)

        # endregion

        #region Class Description

        class_desc_frame = ttk.Frame(class_tab)
        class_desc_frame.grid(row=2, column=6, padx=10, pady=10, sticky='n')

        class_desc = DESC['Class_Tab_Desc'] 

        ttk.Label(class_desc_frame, text=class_desc, wraplength=650, justify='left').grid(row=0, column=0)

        #endregion

        # Generate Button
        class_button = ttk.Button(class_tab, text="Generate", command=self.generate_class)
        class_button.grid(row=4, column=0, columnspan=99, sticky="nsew")

    def generate_class(self):
        class_data = {
            "class": self.class_select.get(),
            "promote": self.class_promote.get(),
            "ranks": [min.get() for min in self.class_min_ranks]
            + [max.get() for max in self.class_max_ranks],
            "stats": [stat.get() for stat in self.class_stats]
        }

        version = self.version.get()
        set_version(version)

        output = get_class_code(class_data)

        self.output_code(output)

    def items_tab(self):

        items_tab = ttk.Frame(self.notebook)
        self.notebook.add(items_tab, text="Items")

        #region Separators
        ttk.Separator(items_tab, orient="horizontal").grid(row=1, column=0, columnspan=99, sticky="ew")
        ttk.Separator(items_tab, orient="horizontal").grid(row=3, column=0, columnspan=99, sticky="ew")
        ttk.Separator(items_tab, orient="vertical").grid(row=2, column=1, rowspan=99, sticky="ns")
        ttk.Separator(items_tab, orient="vertical").grid(row=2, column=3, rowspan=99, sticky="ns")
        #endregion

        #region Item

        item_frame = ttk.Frame(items_tab)
        item_frame.grid(row=0, column=0, padx=10, pady=10, columnspan=99, sticky="nsew")

        ttk.Label(item_frame, text="Item:", font=SECTION_HEADER).grid(row=0, column=0)
        self.item_select = ttk.Combobox(item_frame, values=['All'] + ITEM_LIST, width=25)
        self.item_select.grid(row=0, column=1, padx=10)

        # endregion

        #region Item Data

        item_data_frame = ttk.Frame(items_tab)
        item_data_frame.grid(row=2, column=0, padx=10, pady=10, sticky="n")

        ttk.Label(item_data_frame, text="Weapon Data", font=SECTION_HEADER).grid(row=0, column=0, columnspan=2, pady=5)
        
        self.item_data = {}

        for i, label in enumerate(ITEM_DATA):

            ttk.Label(item_data_frame, text=label.replace('_', ' ')).grid(row=i + 1, column=0, padx=5, sticky="e")

            if label == "Attack_Type":
                self.item_data[label] = ttk.Combobox(item_data_frame, values=["STR", "MAG"], width=5)

            elif label == "Weapon_Rank":
                self.item_data[label] = ttk.Combobox(item_data_frame, values=["SS", "S", "A", "B", "C", "D", "E"], width=5)

            elif label in ["Unlock", "Char_Unlock", "Infinite", "Brave", "Heal"]:
                self.item_data[label] = tk.BooleanVar()
                ttk.Checkbutton(item_data_frame, variable=self.item_data[label]).grid(row=i + 1, column=1)
                continue
            else:
                self.item_data[label] = ttk.Entry(item_data_frame, width=5)

            self.item_data[label].grid(row=i + 1, column=1, pady=1)

        #endregion

        #region Item Stats

        item_stats_frame = ttk.Frame(items_tab)
        item_stats_frame.grid(row=2, column=2, padx=10, pady=10, sticky="n")

        ttk.Label(item_stats_frame, text="Weapon Stats", font=SECTION_HEADER).grid(row=0, column=0, columnspan=2, pady=5)
        self.item_stats = []

        for i, stat in enumerate(ITEM_STATS):
            ttk.Label(item_stats_frame, text=stat.replace('_', ' ')).grid(row=i+1, column=0, padx=5, sticky="e")
            item_stats_entry = ttk.Entry(item_stats_frame, width=5)
            item_stats_entry.grid(row=i+1, column=1, pady=1)
            self.item_stats.append(item_stats_entry)

        #endregion

        #region Item Equip Bonuses

        item_bonus_frame = ttk.Frame(items_tab)
        item_bonus_frame.grid(row=2, column=4, padx=10, pady=10, sticky="n")

        ttk.Label(item_bonus_frame, text="Equip Bonuses", font=SECTION_HEADER).grid(row=0, column=0, columnspan=2, pady=5)

        self.item_bonuses = []

        for i, label in enumerate(ITEM_BONUS):
            ttk.Label(item_bonus_frame, text=label.replace('_', ' ')).grid(row=i + 1, column=0, padx=5, sticky="e")
            item_bonus_entry = ttk.Entry(item_bonus_frame, width=5)
            item_bonus_entry.grid(row=i + 1, column=1, pady=1)
            self.item_bonuses.append(item_bonus_entry)

        # endregion

        # Generate Button
        item_button = ttk.Button(items_tab, text="Generate", command=self.generate_item)
        item_button.grid(row=4, column=0, columnspan=99, sticky="nsew")

    def generate_item(self):
        item_data = {
            "item": self.item_select.get(),
            "data": {
                key: (var.get() if isinstance(var, tk.BooleanVar) else var.get())
                for key, var in self.item_data.items()
            },
            "stats": [stat.get() for stat in self.item_stats],
            "bonuses": [bonus.get() for bonus in self.item_bonuses]
        }

        version = self.version.get()
        set_version(version)

        output = get_item_code(item_data)

        self.output_code(output)

    def database_tab(self):
        database_tab = ttk.Frame(self.notebook)
        self.notebook.add(database_tab, text="Database")
    
        r=0
        c=0
        for cdb, code_id in enumerate(list(CODE_DATABASE)):
            code_id_button = ttk.Button(database_tab, text=code_id, command=lambda cid=code_id: self.generate_database_code(cid), width=15)
            code_id_button.grid(row=r, column=c, padx=5, pady=5)
            if c == 7:
                c = 0
                r += 1
            else:
                c += 1

    def generate_database_code(self, sel_code):
        version = self.version.get()
        set_version(version)
    
        keybinds_data = {
            "controller": self.controller.get(),
            "keys": [key.get() for key in self.checkboxes],
        }
    
        key_code = get_keybind_code(keybinds_data)

        desc = CODE_DATABASE[sel_code]['DESC']
    
        code = CODE_DATABASE[sel_code][VERSION]
    
        if 'unknown' in code:
            output = ' '.join([sel_code, code])
        else:
            output = '\n'.join([desc, key_code, code, 'E0000000 80008000'])
    
        self.output_code(output)

    def generate_all(self):
        character_data = {
            "character": self.char_select.get(),
            "class": self.char_class.get(),
            "stats": [stat.get() for stat in self.char_stats],
            "ranks": [rank.get() for rank in self.char_ranks],
            "items": [
                {
                    "item": row[0].get(),
                    "uses": row[1].get(),
                    "forge_name": row[2].get(),
                    "mt": row[3].get(),
                    "hit": row[4].get(),
                    "crit": row[5].get(),
                    "wt": row[6].get(),
                    "forged": row[7].get(),
                    "blessed": row[8].get(),
                }
                for row in self.char_items
            ]
        }

        class_data = {
            "class": self.class_select.get(),
            "promote": self.class_promote.get(),
            "ranks": [min.get() for min in self.class_min_ranks]
            + [max.get() for max in self.class_max_ranks],
            "stats": [stat.get() for stat in self.class_stats]
        }

        item_data = {
            "item": self.item_select.get(),
            "data": {
                key: (var.get() if isinstance(var, tk.BooleanVar) else var.get())
                for key, var in self.item_data.items()
            },
            "stats": [stat.get() for stat in self.item_stats],
            "bonuses": [bonus.get() for bonus in self.item_bonuses]
        }

        keybinds_data = {
            "controller": self.controller.get(),
            "keys": [key.get() for key in self.checkboxes],
        }

        version = self.version.get()
        set_version(version)

        key_code = get_keybind_code(keybinds_data)

        char_out = get_char_code(character_data, key_code)

        cls_out = get_class_code(class_data)

        item_out = get_item_code(item_data)

        output = '\n'.join([char_out, cls_out, item_out])

        self.output_code(output)

In [5]:
# Main

if __name__ == "__main__":
    root = tk.Tk()
    app = CodeGeneratorGUI(root)
    root.mainloop()